# find coauthor between 2 scientists

This notebook shows one example to visualize the relations between two scientist by using google scholar data obtained by [scholarly](https://pypi.org/project/scholarly/)  

In [1]:
%pip install scholarly scholarly[tor] streamlit-agraph pyngrok  httpx==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.2 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 79.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.

# write code to a .py file
streamlit need to run with local .py file

In [6]:
%%writefile app.py
import streamlit as st
from scholarly import scholarly, ProxyGenerator
from streamlit_agraph import agraph, Config, Node, Edge

# Set up proxy to avoid being banned by Google Scholar
# this will slow down the speed of excution
#pg = ProxyGenerator()
#pg.FreeProxies()
#scholarly.use_proxy(pg)

# Function to fetch co-authors for a researcher
def get_coauthors(name):
    try:
        search_query = scholarly.search_author(name)
        author = next(search_query, None)
        if author:
            detailed_author = scholarly.fill(author, sections=["coauthors"])
            coauthors = detailed_author.get("coauthors", [])
            return {coauthor["name"] for coauthor in coauthors}
        else:
            st.warning(f"No author found for the name: {name}")
            return set()
    except Exception as e:
        st.error(f"Error fetching co-authors: {e}")
        return set()

# Streamlit app
def main():
    st.title("Shared Co-authors Flowchart")
    
    # Input for researcher names
    name1 = st.text_input("Enter the first researcher's name")
    name2 = st.text_input("Enter the second researcher's name")

    # Initialize session state for nodes and edges
    if "nodes" not in st.session_state:
        st.session_state.nodes = []
    if "edges" not in st.session_state:
        st.session_state.edges = []

    if st.button("Generate Flowchart"):
        if name1 and name2:
            st.info("Searching for coauthors... This may take a few moments.")
            coauthors1 = get_coauthors(name1)
            coauthors2 = get_coauthors(name2)

            # Find shared co-authors
            shared_coauthors = coauthors1 & coauthors2
            
            # Clear previous nodes and edges
            st.session_state.nodes = [
                Node(id=name1, label=name1, color="blue"),
                Node(id=name2, label=name2, color="red"),
            ]
            st.session_state.edges = []

            # Add shared coauthors
            for coauthor in shared_coauthors:
                st.session_state.nodes.append(Node(id=coauthor, label=coauthor, color="green"))
                st.session_state.edges.append(Edge(source=name1, target=coauthor))
                st.session_state.edges.append(Edge(source=name2, target=coauthor))

            # Add unique coauthors for name1
            for coauthor in coauthors1 - shared_coauthors:
                st.session_state.nodes.append(Node(id=coauthor, label=coauthor, color="lightblue"))
                st.session_state.edges.append(Edge(source=name1, target=coauthor))

            # Add unique coauthors for name2
            for coauthor in coauthors2 - shared_coauthors:
                st.session_state.nodes.append(Node(id=coauthor, label=coauthor, color="pink"))
                st.session_state.edges.append(Edge(source=name2, target=coauthor))

    # Display flowchart
    config = Config(width=1600, height=900, directed=False, physics=True)
    st.subheader("Coauthor Connection Graph")
    agraph(
        nodes=st.session_state.nodes,
        edges=st.session_state.edges,
        config=config,
    )

if __name__ == "__main__":
    main()

Overwriting app.py


# run the app

## ignore this part if you are running this code locally
if you are running this notebook locally, ignore this part. otherwise you need to use ngrok to expose the port to public. a ngrok spi key is needed for this.

In [4]:
# only run this if you are runnning it in kaggle or colab.
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("ngrok_auth_key")


ngrok.set_auth_token(secret_value_0)
public_url = ngrok.connect(8501) # Expose port 8501
print(public_url)

NgrokTunnel: "https://8852-34-78-48-7.ngrok-free.app" -> "http://localhost:8501"                    


## run the code

In [5]:
!streamlit run app.py 




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.78.48.7:8501

^C
  Stopping...
